<a href="https://colab.research.google.com/github/Guliko24/NetZoo_network/blob/main/GRN_13Laplacian_matrix_with_PyGSP_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# This is a block of code working with single files from MCF7 and MDA-MB-231 cell lines



In [1]:
#import all the packages as needed
import pandas as pd
import networkx as nx

In [2]:
# prompt: let's load Gdrive onto notebook

from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
# Navigate to your Google Drive files
%cd /content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets


In [ ]:
#let's read the MCF7_TF_Genes dataset and make changes
df_MCF7_TF_Genes = pd.read_csv('/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ACH-000019_TF_vs_Genes_total expression.csv', index_col=0)

In [ ]:
#let's do initial data assessment
#df_MCF7_TF_Genes.head()


In [ ]:
#df_MCF7_TF_Genes.info()


In [ ]:
#print(df_MCF7_TF_Genes.describe())

In [ ]:
#print(df_MCF7_TF_Genes.isnull().sum())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#plt.figure(figsize=(10, 8))
#sns.heatmap(df_MCF7_TF_Genes.iloc[:, 1:], cmap='viridis')  # Skipping the first column if it's TF names
#plt.title('Heatmap of Gene Expression by TF')
#plt.show()


In [ ]:
import numpy as np
import pandas as pd

def normalize_matrix(matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize the values in the matrix to a range of [0, 1].
    """
    normalized_matrix = (matrix - matrix.min().min()) / (matrix.max().max() - matrix.min().min())
    return normalized_matrix

def filter_by_threshold(matrix: pd.DataFrame, threshold: float, mode: str = 'greater') -> pd.DataFrame:
    """
    Filter the matrix based on a threshold.
    Parameters:
        matrix: The TF vs Genes interaction matrix.
        threshold: The threshold value for filtering.
        mode: 'greater' to keep values greater than threshold, 'less' to keep values less than threshold.
    """
    if mode == 'greater':
        return matrix[matrix > threshold].fillna(0)
    elif mode == 'less':
        return matrix[matrix < threshold].fillna(0)
    else:
        raise ValueError("Mode must be either 'greater' or 'less'")

def binarize_matrix(matrix: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Binarize the matrix based on a threshold.
    Values greater than or equal to the threshold become 1, otherwise 0.
    """
    binary_matrix = (matrix >= threshold).astype(int)
    return binary_matrix

def rank_interactions(matrix: pd.DataFrame, top_n: int = 5) -> pd.DataFrame:
    """
    Rank the top N interactions for each transcription factor.
    Parameters:
        matrix: The TF vs Genes interaction matrix.
        top_n: Number of top interactions to return for each TF.
    """
    ranked_interactions = pd.DataFrame()
    for tf in matrix.index:
        top_genes = matrix.loc[tf].nlargest(top_n)
        ranked_interactions = pd.concat([ranked_interactions, top_genes], axis=1)
    return ranked_interactions.T

def aggregate_interactions(matrix: pd.DataFrame, axis: int = 0) -> pd.Series:
    """
    Aggregate interaction strengths.
    Parameters:
        axis: 0 to aggregate across genes (per TF), 1 to aggregate across TFs (per Gene).
    """
    return matrix.sum(axis=axis)

def construct_interaction_network(matrix: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Construct an interaction network by keeping only interactions above a threshold.
    Returns a DataFrame representing edges in the network.
    """
    filtered_matrix = filter_by_threshold(matrix, threshold, mode='greater')
    edges = []
    for tf in filtered_matrix.index:
        for gene in filtered_matrix.columns:
            if filtered_matrix.loc[tf, gene] > 0:
                edges.append((tf, gene, filtered_matrix.loc[tf, gene]))
    network_df = pd.DataFrame(edges, columns=['TF', 'Gene', 'Interaction_Strength'])
    return network_df

def split_positive_negative_matrices(matrix: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Split the matrix into two DataFrames: one for positive values and one for negative values.
    Ensure that the matrix values are numeric by coercing errors and replacing non-numeric values with NaN.
    Retain the index and columns in both matrices.
    """
    matrix_numeric = matrix.apply(pd.to_numeric, errors='coerce')
    positive_matrix = matrix_numeric.where(matrix_numeric > 0).fillna(0)
    negative_matrix = matrix_numeric.where(matrix_numeric < 0).fillna(0)
    positive_matrix.index = matrix.index
    positive_matrix.columns = matrix.columns
    negative_matrix.index = matrix.index
    negative_matrix.columns = matrix.columns
    return positive_matrix, negative_matrix

def check_repeated_indices_columns(matrix: pd.DataFrame):
    """
    Check for repeated transcription factors (rows) and genes (columns) in the matrix.
    """
    repeated_rows = matrix.index[matrix.index.duplicated()].unique()
    repeated_columns = matrix.columns[matrix.columns.duplicated()].unique()

    print("Repeated Rows (TFs):")
    if len(repeated_rows) > 0:
        print(repeated_rows)
    else:
        print("No repeated rows found.")

    print("\nRepeated Columns (Genes):")
    if len(repeated_columns) > 0:
        print(repeated_columns)
    else:
        print("No repeated columns found.")



In [ ]:
# Example Usage
# Read the TF vs Genes matrix from a CSV file
#df_MCF7_TF_Genes = pd.read_csv('/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ACH-000019_TF_vs_Genes_total expression.csv', index_col=0)

# Normalize the matrix
#normalized_df = normalize_matrix(df_MCF7_TF_Genes)

# Filter interactions greater than 0.5
#filtered_df = filter_by_threshold(df_MCF7_TF_Genes, threshold=0.5, mode='greater')

# Binarize the matrix with a threshold of 0.5
#binary_df = binarize_matrix(df_MCF7_TF_Genes, threshold=0.5)

# Rank top 2 interactions for each TF
#ranked_df = rank_interactions(df_MCF7_TF_Genes, top_n=2)

# Aggregate interactions across genes (per TF)
#aggregated_series = aggregate_interactions(df_MCF7_TF_Genes, axis=1)

# Construct interaction network with a threshold of 0.5
#network_df = construct_interaction_network(df_MCF7_TF_Genes, threshold=0.5)

# Split the matrix into positive and negative interaction matrices
positive_df, negative_df = split_positive_negative_matrices(df_MCF7_TF_Genes)

In [ ]:
positive_df.head()

In [ ]:
negative_df

Let's check each dataframes (positive and negative) for:

1) distribution of values:

  remove zero interactions
  may be create subnodes where there are more genes per TF
  may be create subnodes where there are more TFs per gene

2) non-zero interactions

3) visualization


In [ ]:
# Check summary statistics for positive and negative matrices
print("Positive Matrix Summary:")
print(positive_df.describe())
print("\nNegative Matrix Summary:")
print(negative_df.describe())

# Count non-zero interactions in positive and negative matrices
positive_nonzero_count = (positive_df > 0).sum().sum()
negative_nonzero_count = (negative_df < 0).sum().sum()

print(f"\nNumber of non-zero interactions in Positive Matrix: {positive_nonzero_count}")
print(f"Number of non-zero interactions in Negative Matrix: {negative_nonzero_count}")

# Visualize the matrices using heatmaps
import matplotlib.pyplot as plt
import seaborn as sns

# Positive Matrix Heatmap
#plt.figure(figsize=(10, 6))
#sns.heatmap(positive_df, cmap="YlGnBu", cbar=True)
#plt.title("Positive Interactions Heatmap")
#plt.show()

# Negative Matrix Heatmap
#plt.figure(figsize=(10, 6))
#sns.heatmap(negative_df, cmap="coolwarm", cbar=True)
#plt.title("Negative Interactions Heatmap")
#plt.show()


In [ ]:
#lets check whether there is a repetition of rows or columns
# Check for repeated rows (TFs) and columns (Genes)
check_repeated_indices_columns(positive_df)
check_repeated_indices_columns(negative_df)

In [ ]:
# Flatten the dataframe into a series with TF-Gene pair as the index
ranked_positive_df = positive_df.stack().sort_values(ascending=False)

# Get the top 200 interactions
top_300_positive_df = ranked_positive_df.head(300)
print(top_300_positive_df)


In [ ]:
# Flatten the dataframe into a series with TF-Gene pair as the index
ranked_negative_df = negative_df.stack().sort_values(ascending=True)

# Get the top 200 interactions
top_300_negative_df = ranked_negative_df.head(300)
print(top_300_negative_df)


In [ ]:
# Rename the columns of the positive and negative dataframes
top_300_positive_df = top_300_positive_df.reset_index()
top_300_positive_df.columns = ["Source", "Target", "Edge weight"]

top_300_negative_df = top_300_negative_df.reset_index()
top_300_negative_df.columns = ["Source", "Target", "Edge weight"]

# Display the updated dataframes
print("Top 300 Positive Interactions:")
print(top_300_positive_df.head())

print("\nTop 300 Negative Interactions:")
print(top_300_negative_df.head())


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from networkx.algorithms import community



# Initialize directed graph for positive interactions
G_positive = nx.DiGraph()

# Add edges from the dataframe to the directed graph
for _, row in top_300_positive_df.iterrows():
    G_positive.add_edge(row['Source'], row['Target'], weight=row['Edge weight'])

# Set a layout for the graph
pos = nx.spring_layout(G_positive, k=0.8, iterations=100)

# Separate nodes based on Source and Target
source_nodes = set(top_300_positive_df['Source'])
target_nodes = set(top_300_positive_df['Target'])

# Draw the main graph
plt.figure(figsize=(12, 10))
nx.draw(
    G_positive, pos, with_labels=True, node_size=200, nodelist=source_nodes,
    node_color="pink", alpha=0.7, font_size=8, edge_color="darkgrey"
)
nx.draw(
    G_positive, pos, with_labels=True, node_size=200, nodelist=target_nodes,
    node_color="blue", alpha=0.6, font_size=8, edge_color="darkgrey"
)
plt.title("Main Graph with TFs (Pink) and Genes (Blue)")
plt.show()

# Calculate communities
communities = community.greedy_modularity_communities(G_positive)
# Create subplots for communities
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

for idx, comm in enumerate(communities[:2]):  # First two communities
    subgraph = G_positive.subgraph(comm)
    sub_pos = nx.spring_layout(subgraph, k=0.8)  # Separate layout for each community

    # Separate nodes into TFs and genes for this subgraph
    sub_source_nodes = [node for node in comm if node in source_nodes]
    sub_target_nodes = [node for node in comm if node in target_nodes]

    # Draw source nodes (TFs) in pink
    nx.draw(
        subgraph, sub_pos, ax=axes[idx], with_labels=True, node_size=200,
        nodelist=sub_source_nodes, node_color="pink", alpha=0.7, font_size=8
    )

    # Draw target nodes (genes) in blue
    nx.draw(
        subgraph, sub_pos, ax=axes[idx], with_labels=True, node_size=200,
        nodelist=sub_target_nodes, node_color="blue", alpha=0.6, font_size=8
    )

    axes[idx].set_title(f"Community {idx + 1}")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd


# Initialize directed graph for positive interactions
G_negative = nx.DiGraph()

# Add edges from the dataframe to the directed graph
for _, row in top_300_negative_df.iterrows():
    G_negative.add_edge(row['Source'], row['Target'], weight=row['Edge weight'])

# Set a layout for the graph
pos = nx.spring_layout(G_negative, k=0.6, iterations=30)

# Separate nodes based on Source and Target
source_nodes = set(top_300_negative_df['Source'])
target_nodes = set(top_300_negative_df['Target'])

# Draw the source nodes in pink
nx.draw(G_negative, pos, with_labels=True, node_size=1000, nodelist=source_nodes, node_color="orange", font_size=8, edge_color='darkgrey')

# Draw the target nodes in blue, without overriding the previously drawn source nodes
nx.draw(G_negative, pos, with_labels=True, node_size=1000, nodelist=target_nodes, node_color="green", alpha=0.6, font_size=8, edge_color='darkgrey')

# Draw edge labels to show weights with smaller font size
#edge_labels = nx.get_edge_attributes(G_negative, 'weight')
#nx.draw_networkx_edge_labels(G_negative, pos, edge_labels=edge_labels, font_size=8)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from networkx.algorithms import community

# Initialize directed graph for negative interactions
G_negative = nx.DiGraph()

# Add edges from the dataframe to the directed graph
for _, row in top_300_negative_df.iterrows():
    G_negative.add_edge(row['Source'], row['Target'], weight=row['Edge weight'])

# Set a layout for the graph
pos = nx.spring_layout(G_negative, k=0.6, iterations=30)

# Separate nodes based on Source and Target
source_nodes = set(top_300_negative_df['Source'])
target_nodes = set(top_300_negative_df['Target'])

# Draw the main graph
plt.figure(figsize=(10, 8))
nx.draw(
    G_negative, pos, with_labels=True, node_size=1000, nodelist=source_nodes,
    node_color="orange", alpha=0.7, font_size=8, edge_color="darkgrey"
)
nx.draw(
    G_negative, pos, with_labels=True, node_size=1000, nodelist=target_nodes,
    node_color="green", alpha=0.6, font_size=8, edge_color="darkgrey"
)
plt.title("Main Graph with TFs (Orange) and Genes (Green)")
plt.show()

# Calculate communities for the negative graph
communities_negative = community.greedy_modularity_communities(G_negative)

# Create subplots for communities
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

for idx, comm in enumerate(communities_negative[:2]):  # First two communities
    subgraph = G_negative.subgraph(comm)
    sub_pos = nx.spring_layout(subgraph, k=0.8)  # Separate layout for each community

    # Separate nodes into TFs and genes for this subgraph
    sub_source_nodes = [node for node in comm if node in source_nodes]
    sub_target_nodes = [node for node in comm if node in target_nodes]

    # Draw source nodes (TFs) in orange
    nx.draw(
        subgraph, sub_pos, ax=axes[idx], with_labels=True, node_size=1000,
        nodelist=sub_source_nodes, node_color="orange", alpha=0.7, font_size=8
    )

    # Draw target nodes (genes) in green
    nx.draw(
        subgraph, sub_pos, ax=axes[idx], with_labels=True, node_size=1000,
        nodelist=sub_target_nodes, node_color="green", alpha=0.6, font_size=8
    )

    axes[idx].set_title(f"Community {idx + 1}")

plt.tight_layout()
plt.show()


I have attempted using Kamada and Planar layouts and it did not yield any explainable or reasonable plots

In [ ]:
#Save the graph in formats like GraphML or GEXF for visualization in tools like Gephi.
nx.write_graphml(G_positive, "positive_interactions.graphml")
nx.write_graphml(G_negative, "negative_interactions.graphml")

# Let's install PyGPS (suggested by Ortega) and use it for signal processing:

In [ ]:
pip install pygsp


# **1. Convert NetworkX Graph to PyGSP Graph**

In [ ]:
#I decided to use PyGSP package so few adjustments will be required
import networkx as nx
import pygsp as pg
from networkx import to_scipy_sparse_array

# Extract adjacency matrix from NetworkX graph
# Extract adjacency matrix from NetworkX graph as a sparse array
G_positive.remove_edges_from(nx.selfloop_edges(G_positive))##### remove the loop here so now I have 299 edges
adjacency_matrix_pos = to_scipy_sparse_array(G_positive, weight=" Edge weight")

# Create PyGSP graph
gsp_graph_pos= pg.graphs.Graph(adjacency_matrix_pos)

# Display basic graph info
print(f"PyGSP Graph Positive with {gsp_graph_pos.N} nodes and {gsp_graph_pos.Ne} edges.")


In [ ]:
# Extract adjacency matrix from NetworkX graph for Negative graph
# Extract adjacency matrix from NetworkX graph as a sparse array
adjacency_matrix_neg = to_scipy_sparse_array(G_negative, weight=" Edge weight")

# Create PyGSP graph
gsp_graph_neg= pg.graphs.Graph(adjacency_matrix_neg)

# Display basic graph info
print(f"PyGSP Graph Negative with {gsp_graph_neg.N} nodes and {gsp_graph_neg.Ne} edges.")


since our positive graph has loops, i.e. gene strongly interacting with itself (may be acts as a dimer?), I decided to remove that loop for further analysis.


---




# **2.Compute the Laplacian Matrix**


In [ ]:
# Compute Laplacian (default is combinatorial)
gsp_graph_pos.compute_laplacian(lap_type="combinatorial")
laplacian = gsp_graph_pos.L

print("Laplacian Matrix Pos:")
print(laplacian.toarray())  # Display dense matrix for small graphs


In [ ]:
# Compute Laplacian (default is combinatorial)
gsp_graph_neg.compute_laplacian(lap_type="combinatorial")
laplacian = gsp_graph_neg.L

print("Laplacian Matrix Neg:")
print(laplacian.toarray())  # Display dense matrix for small graphs


# 3. Assign Graph Signal
Assign RNa-seq data as the graph signal

In [ ]:
import pandas as pd

# Load RNA-seq data
rna_seq_data = pd.read_csv("/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/MCF_RNAseq_countsGSE208731.csv")  # Replace with your RNA-seq file
# Columns: gene, expression

# Extract genes from graphs
positive_genes = set(G_positive.nodes)
negative_genes = set(G_negative.nodes)
all_genes_in_graph = positive_genes.union(negative_genes)

# Filter RNA-seq data to include only relevant genes
filtered_rna_seq_data = rna_seq_data[rna_seq_data['Gene'].isin(all_genes_in_graph)]

# Verify the alignment
missing_genes = all_genes_in_graph - set(filtered_rna_seq_data['Gene'])
if missing_genes:
    print(f"Warning: The following genes are missing in the RNA-seq data: {missing_genes}")
else:
    print("All graph genes have corresponding RNA-seq data.")



In [ ]:
#average the expression data based on experimental coditions
import pandas as pd



# Averaging specific column groups and renaming them
column_groups = {
    "MCF7_2D_exp1": [1, 2, 3],      # Columns 1, 2, 3
    "MCF7_3D_exp1": [4, 5, 6],      # Columns 4, 5, 6
    "MCF7_2D_exp2": [7, 8, 9, 10, 11, 12],  # Columns 7 to 12
    "MCF7_3D_exp2": [13, 14, 15, 16, 17, 18]  # Columns 13 to 18
}

# Initialize a new DataFrame with the 'gene' column
averaged_data = filtered_rna_seq_data[['Gene']].copy()

# Compute averages for each group and add to the new DataFrame
for new_column, indices in column_groups.items():
    averaged_data[new_column] = rna_seq_data.iloc[:, indices].mean(axis=1).round(2)

# Save or inspect the resulting DataFrame
print(averaged_data.head())
averaged_data.to_csv("averaged_rna_seq_data.csv", index=False)


In [ ]:
import networkx as nx
import pandas as pd

#Load RNA-seq data
averaged_rna_seq_data = pd.read_csv("/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/averaged_rna_seq_data.csv")  # Replace with your RNA-seq file



# Ensure the RNA-seq data has a 'Gene' column and averaged columns (e.g., MCF7_2D_exp1, etc.)
# Create a mapping of genes to expression values for each condition

# Extract nodes for G_positive and G_negative
positive_genes = set(G_positive.nodes)
negative_genes = set(G_negative.nodes)

# Filter RNA-seq data for genes in G_positive and G_negative
positive_rna_seq = averaged_rna_seq_data[averaged_rna_seq_data['Gene'].isin(positive_genes)]
negative_rna_seq = averaged_rna_seq_data[averaged_rna_seq_data['Gene'].isin(negative_genes)]

# Create dictionaries for each graph and each condition
expression_dicts_positive = {
    condition: dict(zip(positive_rna_seq['Gene'], positive_rna_seq[condition]))
    for condition in averaged_rna_seq_data.columns[1:]  # Skip 'gene' column
}

expression_dicts_negative = {
    condition: dict(zip(negative_rna_seq['Gene'], negative_rna_seq[condition]))
    for condition in averaged_rna_seq_data.columns[1:]  # Skip 'gene' column
}

# Assign expression values to nodes in G_positive for a specific condition (example: MCF7_2D_exp1)
selected_condition = "MCF7_2D_exp1"  # Replace with the condition you want to use
nx.set_node_attributes(G_positive, expression_dicts_positive[selected_condition], name='expression')
nx.set_node_attributes(G_negative, expression_dicts_negative[selected_condition], name='expression')

# Verify attributes are set correctly
print("G_positive node attributes:")
for node, data in G_positive.nodes(data=True):
    print(node, data)

print("\nG_negative node attributes:")
for node, data in G_negative.nodes(data=True):
    print(node, data)


In [ ]:
import numpy as np
import pygsp as pg

# Step 1: Load or define adjacency matrices
# Example: Replace with your actual adjacency matrices
A_pos = np.load("adjacency_positive.npy")  # Positive interaction adjacency matrix
A_neg = np.load("adjacency_negative.npy")  # Negative interaction adjacency matrix

# Step 2: Create graphs using PyGSP
graph_pos = pg.graphs.Graph(A_pos)
graph_neg = pg.graphs.Graph(A_neg)

# Compute Laplacians
graph_pos.compute_laplacian('combinatorial')  # Standard Laplacian for positive graph
graph_neg.compute_laplacian('combinatorial')  # Standard Laplacian for negative graph

# Step 3: Define a graph signal (e.g., gene expression for genes)
# Example: Random signals; replace with real gene expression data
num_genes = A_pos.shape[1]
signal = np.random.rand(num_genes)  # Replace with actual signals for genes

# Step 4: Compute signal variation
L_pos = graph_pos.L  # Laplacian matrix for positive graph
L_neg = graph_neg.L  # Laplacian matrix for negative graph

# Signal variation: f^T L f
variation_pos = signal.T @ L_pos @ signal
variation_neg = signal.T @ L_neg @ signal

print(f"Signal Variation - Positive Graph: {variation_pos}")
print(f"Signal Variation - Negative Graph: {variation_neg}")

# Step 5: Clustering (optional)
# Use spectral decomposition for clustering
eigvals_pos, eigvecs_pos = np.linalg.eigh(L_pos)  # Eigenvalues and eigenvectors
eigvals_neg, eigvecs_neg = np.linalg.eigh(L_neg)

# Example: Clustering using the smallest non-zero eigenvectors
from sklearn.cluster import KMeans

# Number of clusters
num_clusters = 3
kmeans_pos = KMeans(n_clusters=num_clusters)
clusters_pos = kmeans_pos.fit_predict(eigvecs_pos[:, 1:num_clusters+1])  # Positive graph clusters

kmeans_neg = KMeans(n_clusters=num_clusters)
clusters_neg = kmeans_neg.fit_predict(eigvecs_neg[:, 1:num_clusters+1])  # Negative graph clusters

print(f"Clusters for Positive Graph: {clusters_pos}")
print(f"Clusters for Negative Graph: {clusters_neg}")
